# Import Modules

In [1]:
import os
import pandas as pd
from credentials import *
import snowflake.connector

c:\Users\ASANTE\miniconda3\envs\SNOWFLAKE\lib\site-packages\snowflake\connector\options.py:108: UserWarning: You have an incompatible version of 'pyarrow' installed (12.0.1), please install a version that adheres to: 'pyarrow<10.1.0,>=10.0.1; extra == "pandas"'
  warn_incompatible_dep(
Failed to import ArrowResult. No Apache Arrow result set format can be used. ImportError: DLL load failed while importing arrow_iterator: The specified procedure could not be found.


# Helper Functions

In [2]:
def create_wh(name: str, size: str, initially_suspended: str, statement_timeout_in_seconds: int, comment: str):
    """
    Creates a new virtual warehouse in the system
    
    https://docs.snowflake.com/en/sql-reference/sql/create-warehouse

    Parameters

    name: str
        Identifier for the virtual warehouse; must be unique for your account.

    size: str
        Specifies the size of the virtual warehouse. The size determines the amount of compute resources in each 
        cluster in the warehouse and, therefore, the number of credits consumed while the warehouse is running.

    initially_suspended: str
        Specifies whether to automatically resume a warehouse when a SQL statement (e.g. query) is submitted to it.

    statement_timeout_in_seconds: int
        Object parameter that specifies the time, in seconds, after which a running SQL statement (query, DDL, DML, etc.) is canceled by the system.

    comment: str
        Specifies a comment for the warehouse.
        
    """
    try:
        conn = snowflake.connector.connect(user=USERNAME, password=PASSWORD, account=ACCOUNT_ID)
        create_warehouse_query = f"""CREATE WAREHOUSE IF NOT EXISTS {name}
                                     WAREHOUSE_SIZE = {size}
                                     INITIALLY_SUSPENDED = {initially_suspended}
                                     STATEMENT_TIMEOUT_IN_SECONDS = {statement_timeout_in_seconds}
                                     COMMENT = '{comment}'
                                  """
        conn.cursor().execute(command=create_warehouse_query)
        conn.close()
        
        return f"WAREHOUSE '{name}' with size '{size}' was created successfully!"
    
    except snowflake.connector.errors.ProgrammingError as e:
        return f"Error: {e}"
    
    
def create_db(name: str, comment: str):
    """
    Creates a new database in the system

    https://docs.snowflake.com/en/sql-reference/sql/create-database

    Parameters

    name: str
        Specifies the identifier for the database; must be unique for your account.

    comment: str
        Specifies a comment for the database.

    """
    try:
        conn = snowflake.connector.connect(user=USERNAME, password=PASSWORD, account=ACCOUNT_ID)
        create_database_query = f"""CREATE DATABASE IF NOT EXISTS {name}
                                    COMMENT = '{comment}'
                                 """
        conn.cursor().execute(command=create_database_query)
        conn.close()
        
        return f"DATABASE '{name}' was created successfully!"
    
    except snowflake.connector.errors.ProgrammingError as e:
        return f"Error: {e}"
    
    
def create_schema(db_name: str, name: str, comment: str):
    """
    Creates a new schema in the current database.

    https://docs.snowflake.com/en/sql-reference/sql/create-schema

    Parameters

    db_name: str
        Specifies database for the schema

    name: str
        Specifies the identifier for the schema; must be unique for the database in which the schema is created.

    comment: str
        Specifies a comment for the schema.

    """
    try:
        conn = snowflake.connector.connect(user=USERNAME, password=PASSWORD, account=ACCOUNT_ID, database=db_name)
        create_schema_query = f"""CREATE SCHEMA IF NOT EXISTS {name}
                                  COMMENT = '{comment}'
                               """
        conn.cursor().execute(command=create_schema_query)
        conn.close()

        return f"SCHEMA '{name}' was created successfully!"
    
    except snowflake.connector.errors.ProgrammingError as e:
        return f"Error: {e}"
    

def create_table(db_name: str, schema_name: str, name: str, comment: str):
    """
    Creates a new table in the current/specified schema or replaces an existing table.

    https://docs.snowflake.com/en/sql-reference/sql/create-table

    Parameters

    db_name: str
        Specifies database for the schema

    schema_name: str
        Specifies the identifier for the schema; must be unique for the database in which the schema is created.

    name: str
        Specifies the identifier (i.e. name) for the table; must be unique for the schema in which the table is created.

    comment: str
        Specifies a comment for the schema.

    """
    try:
        conn = snowflake.connector.connect(user=USERNAME, password=PASSWORD, account=ACCOUNT_ID, database=db_name, schema=schema_name)
        create_table_query = f"""CREATE TABLE IF NOT EXISTS {name} (
                                                                     trip_id VARCHAR,
                                                                     duration VARCHAR,
                                                                     start_time VARCHAR,
                                                                     end_time VARCHAR,
                                                                     start_station VARCHAR,
                                                                     start_lat VARCHAR,
                                                                     start_lon VARCHAR,
                                                                     end_station VARCHAR,
                                                                     end_lat VARCHAR,
                                                                     end_lon VARCHAR,
                                                                     bike_id VARCHAR,
                                                                     plan_duration VARCHAR,
                                                                     trip_route_category STRING,
                                                                     passholder_type STRING,
                                                                     bike_type STRING
                                                                    )
                         
                                    COMMENT = '{comment}'
                                """
        conn.cursor().execute(command=create_table_query)
        conn.close()

        return f"TABLE '{name}' was created successfully!"
    
    except snowflake.connector.errors.ProgrammingError as e:
        return f"Error: {e}"
    
    
def create_stage(db_name: str, schema_name: str, name: str, comment: str):
    """
    Creates a new named internal or external stage to use for loading data from files into Snowflake tables and unloading data from tables into files

    https://docs.snowflake.com/en/sql-reference/sql/create-stage

    Parameters

    db_name: str
        Specifies database for the schema

    schema_name: str
        Specifies the identifier for the schema; must be unique for the database in which the schema is created.

    name: str
        Specifies the identifier for the stage; must be unique for the schema in which the stage is created.

    comment: str
        Specifies a comment for the stage.

    """
    try:
        conn = snowflake.connector.connect(user=USERNAME, password=PASSWORD, account=ACCOUNT_ID, database=db_name, schema=schema_name)
        create_stage_query = f"""CREATE STAGE IF NOT EXISTS {name}
                                 DIRECTORY = (ENABLE = TRUE)
                                 FILE_FORMAT = (TYPE = 'CSV')
                                 ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE')
                                 COPY_OPTIONS = (ON_ERROR='abort_statement')
                                 COMMENT = '{comment}'
                              """
        conn.cursor().execute(create_stage_query)
        conn.close()

        return f"STAGE '{name}' was created successfully!"
    
    except snowflake.connector.errors.ProgrammingError as e:
        return f"Error: {e}"

# Create Data Warehouse

In [3]:
warehouse_name = 'MY_DATA_WH'
warehouse_size = 'XSMALL'
initially_suspended = 'TRUE'
statement_timeout = 300
warehouse_comment = 'MY_DATA_WH'

create_wh(warehouse_name, warehouse_size, initially_suspended, statement_timeout, warehouse_comment)

"WAREHOUSE 'MY_DATA_WH' with size 'XSMALL' was created successfully!"

# Create Database

In [4]:
database_name = 'Metro_Bike_Share'
database_comment = 'Metro Bike Share trip data'

create_db(database_name, database_comment)

"DATABASE 'Metro_Bike_Share' was created successfully!"

# Create Schema

In [5]:
schema_name = 'My_Bike_Share_Schema'
schema_comment = 'Metro Bike Share trip data'

create_schema(database_name, schema_name, schema_comment)

"SCHEMA 'My_Bike_Share_Schema' was created successfully!"

# Create Table

In [6]:
table_name = "Bike_Share"
tbl_comment = "Metro Bike Share trip data"

create_table(database_name, schema_name, table_name, tbl_comment)

"TABLE 'Bike_Share' was created successfully!"

# Create Stage

In [7]:
internal_stage_name = "la_metro"
stage_comment = "la metro bikeshare stage"

create_stage(database_name, schema_name, internal_stage_name, stage_comment)

"STAGE 'la_metro' was created successfully!"

# Upload Files To Stage

In [8]:
def upload_file_to_stage(db_name: str, schema_name: str, full_file_path: str, stage_name: str, filename: str):
    """
    Uploads (i.e. stages) data files from a local directory/folder on a client machine to one of the following Snowflake stages:
    Named internal stage, Internal stage for a specified table or Internal stage for the current user.

    https://docs.snowflake.com/en/sql-reference/sql/put

    Parameters

    db_name: str
        Specifies database for the schema

    schema_name: str
        Specifies the identifier for the schema; must be unique for the database in which the schema is created.

    full_file_path: str
        full path to the file

    stage_name: str
        Specifies the identifier for the stage; must be unique for the schema in which the stage is created.

    filename: str
        name of the file(s) to upload

    """
    try:
        conn = snowflake.connector.connect(user=USERNAME, password=PASSWORD, account=ACCOUNT_ID, database=db_name, schema=schema_name)
        upload_file_query = f"""PUT file://{full_file_path} @{stage_name}"""

        conn.cursor().execute(upload_file_query)
        conn.close()

        return f"'{filename}' was successfully uploaded!"
    
    except snowflake.connector.errors.ProgrammingError as e:
        return f"Error: {e}"

In [9]:
file_to_upload = 'metro-trips-2023-q1.csv'
stage_name = 'la_metro'
full_file_path = os.path.join(os.getcwd(), 'Datasets', 'MetroBikeShare', 'metro-trips-2023-q1.csv')

upload_file_to_stage(database_name, schema_name, full_file_path, stage_name, f'{file_to_upload}')

"'metro-trips-2023-q1.csv' was successfully uploaded!"